<a href="https://colab.research.google.com/github/vanezp01/datalabs/blob/main/Trading_buddy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# import libraries
import yfinance as yf
import pandas as pd
import numpy as np
from ta.trend import MACD, SMAIndicator
from ta.momentum import RSIIndicator
import streamlit as st
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

In [5]:
# Function to fetch data
def fetch_data(ticker):
    end_date = datetime.today()
    start_date = end_date - timedelta(days=60)
    data = yf.download(ticker, start=start_date, end=end_date)
    return data

In [6]:
# Function to compute indicators
def compute_indicators(data):
    data['SMA_20'] = SMAIndicator(data['Close'], window=20).sma_indicator()
    data['SMA_50'] = SMAIndicator(data['Close'], window=50).sma_indicator()
    data['RSI'] = RSIIndicator(data['Close'], window=14).rsi()
    macd = MACD(data['Close'])
    data['MACD'] = macd.macd()
    data['MACD_signal'] = macd.macd_signal()
    return data

In [7]:
# Basic support/resistance estimation
def support_resistance(data):
    recent = data[-20:]
    support = recent['Low'].min()
    resistance = recent['High'].max()
    return support, resistance

In [8]:
# Predict next week's high/low (simple)
def predict_week_range(data):
    last_close = data['Close'].iloc[-1]
    volatility = data['Close'].rolling(window=5).std().iloc[-1]
    predicted_high = last_close + (volatility * 1.5)
    predicted_low = last_close - (volatility * 1.5)
    return round(predicted_low, 2), round(predicted_high, 2)

In [9]:
# Suggest trade levels
def trade_levels(data, support, resistance):
    entry = data['Close'].iloc[-1]
    stop_loss = round(support * 0.98, 2)
    take_profit = round(resistance * 0.98, 2)
    return round(entry, 2), stop_loss, take_profit


In [10]:
# Main web app function
def main():
    st.title("AI Swing Trading Buddy")
    ticker = st.text_input("Enter stock or ETF ticker (e.g., AAPL, SPY):", value="AAPL")

    if st.button("Analyze"):
        data = fetch_data(ticker)
        if data.empty:
            st.error("Failed to retrieve data. Check the ticker.")
            return

        data = compute_indicators(data)
        support, resistance = support_resistance(data)
        predicted_low, predicted_high = predict_week_range(data)
        entry, stop_loss, take_profit = trade_levels(data, support, resistance)

        st.subheader("Summary Table")
        summary = pd.DataFrame({
            'Indicator': ['20-day SMA', '50-day SMA', 'RSI', 'MACD', 'MACD Signal', 'Support', 'Resistance', 'Predicted Low', 'Predicted High', 'Entry', 'Stop Loss', 'Take Profit'],
            'Value': [
                round(data['SMA_20'].iloc[-1], 2),
                round(data['SMA_50'].iloc[-1], 2),
                round(data['RSI'].iloc[-1], 2),
                round(data['MACD'].iloc[-1], 2),
                round(data['MACD_signal'].iloc[-1], 2),
                support,
                resistance,
                predicted_low,
                predicted_high,
                entry,
                stop_loss,
                take_profit
            ]
        })
        st.dataframe(summary)

        st.subheader("Price Chart with Moving Averages")
        fig, ax = plt.subplots(figsize=(10, 4))
        ax.plot(data.index, data['Close'], label='Close')
        ax.plot(data.index, data['SMA_20'], label='SMA 20')
        ax.plot(data.index, data['SMA_50'], label='SMA 50')
        ax.axhline(support, color='green', linestyle='--', label='Support')
        ax.axhline(resistance, color='red', linestyle='--', label='Resistance')
        ax.legend()
        st.pyplot(fig)

if __name__ == "__main__":
    main()

2025-05-31 07:50:08.855 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-31 07:50:08.969 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-05-31 07:50:08.970 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-31 07:50:08.971 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-31 07:50:08.973 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-31 07:50:08.974 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-31 07:50:08.976 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-31 07:50:08.977 Session state does not 